# Load the MNIST dataset

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

# Load recorded events list from .pickle file

In [ ]:
# Set params value for .pkl file name:

fr_ev_th=4
mnist_sym_start=30000
mnist_sym_end=49999
frame_size = 16
# list_np_fr = [[] for i in range(3)]

In [ ]:
import pickle

f_path='mnist_dvs/'
f_name='mnist_'+str(mnist_sym_start)+'_'+str(mnist_sym_end)+'.pkl'
full_path=f_path+f_name
with open(full_path, 'rb') as f:
    events = pickle.load(f)

In [ ]:
print(f_name)
print(len(events))
print(len(events[3]))
# print(type(events[0]))

# Create a list of numpy frames (with crop) on_off/on/off

In [ ]:
# operational with crop and create a list of 3d numpy arrays = num_of_frames x 2D frame

import matplotlib.pyplot as plt
%matplotlib inline

def to_uint8( data ) :
    # maximum pixel
    latch = np.zeros_like( data )
    latch[:] = 255
    # minimum pixel
    zeros = np.zeros_like( data )

    # unrolled to illustrate steps
    d = np.maximum( zeros, data )
    d = np.minimum( latch, d )

    # cast to uint8
    return np.asarray( d, dtype="uint8" )

# num_of_mnist_symbols=6000
original_frame_size=32

# ----------Input parameters:--------
max_num_of_frames=100
# max_num_of_frames=10
frame_size=16
# -----------------------------------

crop_x_min=(original_frame_size-frame_size)//2+1+(original_frame_size-frame_size)%2
crop_x_max=original_frame_size-((original_frame_size-frame_size)//2+1)
crop_y_min=crop_x_min
crop_y_max=crop_x_max

list_np_fr_arr_on_off = []
list_np_fr_arr_on = []
list_np_fr_arr_off = []
# fr_ev_th=8
for mnist_sym in events:
    np_fr_arr_on_off=np.zeros((max_num_of_frames,frame_size,frame_size))
    np_fr_arr_on=np.zeros((max_num_of_frames,frame_size,frame_size))
    np_fr_arr_off=np.zeros((max_num_of_frames,frame_size,frame_size))
#     ev_cnt=[0,0,0]
#     fr_num=[0,0,0]
    ev_cnt=0
    fr_num=0
    for e in mnist_sym:
        x=e[1]-crop_x_min
        y=e[2]-crop_y_min
        if e[1]>=crop_x_min and e[1]<=crop_x_max and e[2]>=crop_y_min and e[2]<=crop_y_max:
#             np_fr_arr_on_off[fr_num[0], (frame_size-1)-y,x]=np_fr_arr_on_off[fr_num[0], (frame_size-1)-y,x]+1
            np_fr_arr_on_off[fr_num, (frame_size-1)-y,x]=np_fr_arr_on_off[fr_num, (frame_size-1)-y,x]+1
            if e[3]==1:
#                 if ev_cnt[1]>=fr_ev_th-1 and fr_num[1]<max_num_of_frames-1:
#                     fr_num[1]+=1
#                     ev_cnt[1]=0
#                 else:
#                     ev_cnt[1]+=1
#                 np_fr_arr_on[fr_num[1], (frame_size-1)-y,x]=np_fr_arr_on[fr_num[1], (frame_size-1)-y,x]+1
                np_fr_arr_on[fr_num, (frame_size-1)-y,x]=np_fr_arr_on[fr_num, (frame_size-1)-y,x]+1
            else:
#                 if ev_cnt[2]>=fr_ev_th-1 and fr_num[2]<max_num_of_frames-1:
#                     fr_num[2]+=1
#                     ev_cnt[2]=0
#                 else:
#                     ev_cnt[2]+=1
#                 np_fr_arr_off[fr_num[2], (frame_size-1)-y,x]=np_fr_arr_off[fr_num[2], (frame_size-1)-y,x]+1
                np_fr_arr_off[fr_num, (frame_size-1)-y,x]=np_fr_arr_off[fr_num, (frame_size-1)-y,x]+1

#             if ev_cnt[0]>=fr_ev_th-1 and fr_num[0]<max_num_of_frames-1:
#                 fr_num[0]+=1
#                 ev_cnt[0]=0
#             else:
#                 ev_cnt[0]+=1
            if ev_cnt>=fr_ev_th-1 and fr_num<max_num_of_frames-1:
                fr_num+=1
                ev_cnt=0
            else:
                ev_cnt+=1
    
#     fr_num[0] += 1
#     fr_num[1] += 1
#     fr_num[2] += 1
    fr_num += 1
    
#     list_np_fr_arr_on_off.append(np_fr_arr_on_off[:fr_num[0],:,:])
#     list_np_fr_arr_on.append(np_fr_arr_on[:fr_num[1],:,:])
#     list_np_fr_arr_off.append(np_fr_arr_off[:fr_num[2],:,:])
    list_np_fr_arr_on_off.append(np_fr_arr_on_off[:fr_num,:,:]/fr_ev_th)
    list_np_fr_arr_on.append(np_fr_arr_on[:fr_num,:,:]/fr_ev_th)
    list_np_fr_arr_off.append(np_fr_arr_off[:fr_num,:,:]/fr_ev_th)


list_np_fr_6000=[list_np_fr_arr_on_off,list_np_fr_arr_on,list_np_fr_arr_off]

# -----Testing parameters:----------    
frame_sample = 0
mnist_sym_relative_ind=0
# ----------------------------------

print(len(list_np_fr_arr_on_off))
print(y_train[mnist_sym_relative_ind])
print(type(list_np_fr_arr_on_off[mnist_sym_relative_ind][frame_sample,:,:]))
print(list_np_fr_arr_on_off[mnist_sym_relative_ind][frame_sample,:,:].shape)
image = list_np_fr_arr_on_off[mnist_sym_relative_ind][frame_sample,:,:]
# plot the sample
fig = plt.figure
plt.imshow(image, cmap='gray')
plt.show()

# Load and plot frame samples from the list of 6000 mnist symbol event frames

In [ ]:
# mnist_sym_start=12000
frame_sample = 2
mnist_sym_relative_ind=2000
print(len(list_np_fr_arr_on_off))
print(y_train[mnist_sym_start+mnist_sym_relative_ind])
print(type(list_np_fr_arr_on[mnist_sym_relative_ind]))
print(list_np_fr_arr_on[mnist_sym_relative_ind].shape)
for frame_sample in range(list_np_fr_arr_on[mnist_sym_relative_ind].shape[0]):
    image = list_np_fr_arr_on[mnist_sym_relative_ind][frame_sample,:,:]
    # plot the sample
    fig = plt.figure
    plt.imshow(image, cmap='gray')
    plt.show()
    print(np.sum(image))

# Save recorded events numpy frame lists to .pickle file

In [ ]:
import pickle

# list_np_fr=[list_np_fr_arr_on_off,list_np_fr_arr_on,list_np_fr_arr_off]
f_path='mnist_dvs/'
f_name='mnist_'+str(mnist_sym_start)+'_'+str(mnist_sym_end)+'_list_np_fr_'+str(frame_size)+'_ev_th_'+str(fr_ev_th)+'.pkl'
full_path=f_path+f_name

print(full_path)
with open(full_path, 'wb') as f:
    pickle.dump(list_np_fr, f)

In [ ]:
# list_np_fr_6000=[list_np_fr_arr_on_off,list_np_fr_arr_on,list_np_fr_arr_off]

# Load recorded events list from .pickle file

In [ ]:
import pickle

mnist_start = 12000
mnist_end = 17999

f_path='./'
f_name='mnist_'+str(mnist_start)+'_'+str(mnist_end)+'_list_np_fr_'+str(frame_size)+'_ev_th_'+str(fr_ev_th)+'.pkl'
full_path=f_path+f_name
with open(full_path, 'rb') as f:
    list_np_fr_6000 = pickle.load(f)
    
    

In [ ]:
list_np_fr_6000[0][869]

In [ ]:
# list_np_fr = [[] for i in range(3)]

mnist_sym_start=0
mnist_sym_end=49999

In [ ]:
mnist_sym_start=0
mnist_sym_end=24000

# for i in range(3):
#     list_np_fr[i] = list_np_fr[i][mnist_sym_start:mnist_sym_end]


In [ ]:
for i in range(3):
    for frames in list_np_fr_6000[i]:
        list_np_fr[i].append(frames)


In [ ]:
len(list_np_fr[0])

In [ ]:
f_path='mnist_dvs/'
f_name='mnist_'+str(mnist_sym_start)+'_'+str(mnist_sym_end)+'_list_np_fr_'+str(frame_size)+'_ev_th_'+str(fr_ev_th)+'.pkl'
full_path=f_path+f_name

print(full_path)
with open(full_path, 'wb') as f:
    pickle.dump(list_np_fr, f)
    
len(list_np_fr[0])

# Load and plot frame samples from the list of 6000 mnist symbol event frames

In [ ]:
# frame_sample = 1
mnist_sym_relative_ind=41999
on_off = 0# 0-both,1-on,2-off
print(y_train[mnist_sym_start+mnist_sym_relative_ind])
# print(len(list_np_fr[1]))

frames_n = list_np_fr[on_off][mnist_sym_relative_ind].shape[0]
fig, axs = plt.subplots(1, frames_n, figsize=(10,6))
image_sum = np.zeros(list_np_fr[on_off][mnist_sym_relative_ind][0,:,:].shape)

for frame_sample in range(frames_n):
    image = list_np_fr[on_off][mnist_sym_relative_ind][frame_sample,:,:] # list_np_fr_arr_on on events only
    if frames_n>1:
        axs[frame_sample].imshow(image, cmap='gray')
        axs[frame_sample].set_title('frame'+str(frame_sample))
    else:
        axs.imshow(image, cmap='gray')
        axs.set_title('frame'+str(frame_sample))
    image_sum += image
# plt.savefig('frames_image10004.png')

plt.figure()
plt.imshow(image_sum)
# plt.savefig('image_sum7.png')


In [ ]:
#  generate single matrix with all events
on_off = 0

list_allevents = []
list_eventsum = []
for indx, mnist_sample in enumerate(list_np_fr[on_off]):
    image_sum = np.zeros(mnist_sample[0,:,:].shape)
    for frame in mnist_sample:
        image_sum += frame
        
    list_allevents.append(image_sum[np.newaxis,:,:])
    list_eventsum.append(np.sum(image_sum))

print(len(list_allevents))

In [ ]:
def dvs_generate_fr_dataset(list_np_fr, on_off = 0, ts = 6):

    traj = np.zeros([ts,2])
    missing_frames_count = 0
    
    dvs_images = []
    trajectories = []
    
    for ind, samples in enumerate(list_np_fr):
        dimim = np.zeros([ts, frame_size, frame_size, 1])
        frames = np.min([ts,samples.shape[0]])#last frame contains less than 'th' events
        dimim[-frames:,:,:,0] = samples[:frames,:,:]

#         if samples.shape[0] < ts:
#             missing_frames_count += 1
        
        dvs_images.append(dimim)
        trajectories.append(traj)
        
    # dvs_images = np.array(dvs_images)[:,:,:,:, np.newaxis]
    dvs_images = np.array(dvs_images)[:,:,:,:]
    trajectories = np.array(trajectories)
    x_dvs = (dvs_images, trajectories)
        
    return x_dvs

def dvs_generate_onoff_lasagna(ds1, ds2):
    ds_onoff = [[],[]]
    ds_onoff[0] = np.stack((ds1[0].squeeze(),ds2[0].squeeze()),axis=4)
    ds_onoff[1] = ds1[1]

    return ds_onoff
#     test_x_dvs_onoff = [[],[]]
#     test_x_dvs_onoff[0] = np.stack((test_x_dvs[0].squeeze(),test_x_dvs_[0].squeeze()),axis=4)
#     # test_x_dvs_onoff[0] = np.stack((test_x_dvs[0].squeeze(),test_x_dvs_[0].squeeze()))
#     test_x_dvs_onoff[1] = test_x_dvs[1]

def dvs_generate_ds(list_np_fr, on_off = 0, ts = 6, mnist_indx = [0,10000], sym_used = [0,10000], n_val = 1000, lasgna = True):
# on_off = 1# 0-both,1-on,2-off,3-on,off
# ts = 3
# first_mnist_indx = 0 # absolute- relative to mnist dataset
# last_mnist_indx = 49999

# used_sym_start = 5000 # relative- samples to use to build dataset
# used_sym_end = 49999
#     val_n = 4000

    y_train_aligned = y_train[mnist_indx[0]:mnist_indx[1]]
    rand_perm = np.random.permutation(range(sym_used[0], sym_used[1]))
    train_ind = rand_perm[0:-n_val]
    test_ind = rand_perm[-n_val:]

    #train data
    list_fr_train = [list_np_fr[on_off[0]][i] for i in train_ind]
    train_x_dvs = dvs_generate_fr_dataset(list_fr_train, on_off[0], ts)
    train_y_dvs = y_train_aligned[train_ind]

    #test data
    list_fr_test = [list_np_fr[on_off[0]][i] for i in test_ind]
    test_x_dvs = dvs_generate_fr_dataset(list_fr_test, on_off[0], ts)
    test_y_dvs = y_train_aligned[test_ind]

    if lasgna:
       #second data set
#        on_off = 2 # 0-both,1-on,2-off,3-on,off
       #train data
        list_fr_train = [list_np_fr[on_off[1]][i] for i in train_ind]
        train_x_dvs_ = dvs_generate_fr_dataset(list_fr_train, on_off[1], ts)
       #test data
        list_fr_test = [list_np_fr[on_off[1]][i] for i in test_ind]
        test_x_dvs_ = dvs_generate_fr_dataset(list_fr_test, on_off[1], ts)
        
        train_x_dvs = dvs_generate_onoff_lasagna(train_x_dvs, train_x_dvs_)
        test_x_dvs = dvs_generate_onoff_lasagna(test_x_dvs, test_x_dvs_)

    return train_x_dvs, train_y_dvs, test_x_dvs, test_y_dvs


In [ ]:
# train_x_dvs_onoff = [[],[]]
# train_x_dvs_onoff[0] = np.stack((train_x_dvs[0].squeeze(),train_x_dvs_[0].squeeze()),axis=4)
# # train_x_dvs_onoff[0] = np.stack((train_x_dvs[0].squeeze(),train_x_dvs_[0].squeeze()))
# train_x_dvs_onoff[1] = train_x_dvs[1]

# test_x_dvs_onoff = [[],[]]
# test_x_dvs_onoff[0] = np.stack((test_x_dvs[0].squeeze(),test_x_dvs_[0].squeeze()),axis=4)
# # test_x_dvs_onoff[0] = np.stack((test_x_dvs[0].squeeze(),test_x_dvs_[0].squeeze()))
# test_x_dvs_onoff[1] = test_x_dvs[1]


In [ ]:
import pickle

fr_ev_th = 2
frame_size = 16
mnist_start = 0
mnist_end = 49999

f_path='./mnist_dvs/'
f_name='mnist_'+str(mnist_start)+'_'+str(mnist_end)+'_list_np_fr_'+str(frame_size)+'_ev_th_'+str(fr_ev_th)+'.pkl'
full_path=f_path+f_name
with open(full_path, 'rb') as f:
    list_np_fr = pickle.load(f)

In [ ]:
# import keras_networks
import importlib
importlib.reload(keras_networks)

ts = 12
mnist_indx = [0,49999]
sym_used = [5000,49999]
n_val = 4000
lasgna = True
on_off = [1,2]

runs = 10
val_accuracy_arr = np.zeros(10)
for i in range(0,runs):
    train_x_dvs, train_y_dvs, test_x_dvs, test_y_dvs = dvs_generate_ds(list_np_fr, on_off, 
                                                                  ts, mnist_indx, sym_used, 
                                                                  n_val, lasgna)

    rnn102 = keras_networks.rnn_model_102(n_timesteps=ts,
                                      lr=1e-3,dropout=0.0,
                                      rnn_type='gru',
                                      rnn_layers = 1,
                                      input_size=(frame_size, frame_size, 2),
                                      conv_fe=False,
                                      rnn_units = 100)
  

    print("Fit model on training data, run ",i)
    history = rnn102.fit(
            train_x_dvs,
            train_y_dvs,
            batch_size=128,
            epochs=10,
            # We pass some validation for
            # monitoring validation loss and metrics
            # at the end of each epoch
            validation_data=(test_x_dvs, test_y_dvs)) #
    
    val_accuracy_arr[i] = history.history['val_sparse_categorical_accuracy'][-1:][0]

In [ ]:
# print(np.mean(val_accuracy_arr), np.std(val_accuracy_arr))
# val_accuracy_arr_4_12 = val_accuracy_arr

data = [val_accuracy_arr_2_24, val_accuracy_arr_2_12, val_accuracy_arr_4_12, val_accuracy_arr_8, val_accuracy_arr_16]
plt.figure()
plt.boxplot(data, labels = ["ev_th = 2\nts=24", "ev_th = 2\nts=12", "ev_th = 4\nts=12", "ev_th = 8\nts=6", "ev_th = 16\nts=3"])
plt.ylabel("accuracy")

plt.savefig('performance_vs_ev-th.png')

In [ ]:
pred102 = rnn102.predict(test_x_dvs_onoff)

plt.figure(figsize=[4,1])
plt.bar(['0', '1' , '2', '3', '4', '5', '6', '7', '8', '9'], height = pred102[110])

# plt.savefig('network_output.png')

In [ ]:
f_path='mnist_dvs/'

symbols_in_files= [[0, 5999], [6000, 11999], [12000, 17999], [18000, 23999], [24000, 29999]]
num_of_events = [[] for i in symbols_in_files]
for i, symbols in enumerate(symbols_in_files):
    
    f_name='mnist_'+str(symbols[0])+'_'+str(symbols[1])+'.pkl'
    full_path=f_path+f_name
    with open(full_path, 'rb') as f:
        events = pickle.load(f)
    
    for sym in events:
        num_of_events[i].append(len(sym))
        
    

In [ ]:
plt.figure()
# plt.hist([num_of_events[0], num_of_events[1], num_of_events[2]], density = True)
plt.hist(num_of_events[0], density = True)

#  plt.figure()
plt.hist(num_of_events[1], alpha=0.6, density=True)

# plt.figure()
plt.hist(num_of_events[2], alpha = 0.6, density=True)

plt.hist(num_of_events[3], alpha = 0.6, density=True)

plt.hist(num_of_events[4], alpha = 0.6, density=True)

In [ ]:
print(np.mean(num_of_events[0]), np.mean(num_of_events[1]), np.mean(num_of_events[2]))
print(np.std(num_of_events[0]), np.std(num_of_events[1]), np.std(num_of_events[2]))
# print(tf.__version__)

In [ ]:
plt.figure()
# plt.hist([num_of_events[0], num_of_events[1], num_of_events[2]], density = True)
# plt.hist(list_eventsum[0:6000], density = True)

#  plt.figure()
# plt.hist(list_eventsum[6000:12000], alpha=0.6, density=True)

# plt.figure()
# plt.hist(list_eventsum[12000:18000], alpha = 0.6, density=True)

# plt.hist(list_eventsum[18000:24000], alpha = 0.6 , density=True)

# plt.hist(list_eventsum[24000:30000], alpha = 0.6, density=True)

# plt.hist(list_eventsum[36000:42000], alpha = 0.6 , density=True)

# plt.hist(list_eventsum[36000:48000], alpha = 0.6 , density=True)

# plt.hist(list_eventsum[48000:54000], alpha = 0.6 , density=True)

# print(np.mean(list_eventsum[36000:48000]))
plt.hist(list_eventsum[5000:10000], alpha = 0.6 , density=True)
# plt.hist(list_eventsum[0:10000], alpha = 0.6 , density=True)
plt.hist(list_eventsum[10000:20000], alpha = 0.6 , density=True)
plt.hist(list_eventsum[20000:30000], alpha = 0.6 , density=True)
plt.hist(list_eventsum[30000:40000], alpha = 0.6 , density=True)
plt.hist(list_eventsum[40000:50000], alpha = 0.6 , density=True)

In [ ]:
list_eventsum[18000:18010]

In [ ]:
list_eventsum_zero = [i==0.0 for i in list_eventsum[24000:36000]]
all(list_eventsum_zero)

In [ ]:
models_path = 'models/'
model_name = 'mnist_dvs_keras_lowres'

rnn102.save(models_path+model_name+'.h5')

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model(models_path+model_name+'.h5')

# Show the model architecture
new_model.summary()


In [ ]:
mnist_img = x_train[36100, :, :]

plt.figure()
plt.imshow(mnist_img)
plt.savefig('mnist_fig_36100.png')